In [1]:
import gym
import mujoco_py

In [2]:
"""The code is adapted from gym"""
import os
import numpy as np
from gym import utils
from gym.envs.mujoco import mujoco_env

import gym


class MyEnv(gym.Env):
    def __init__(self, para, **kwargs):
        super(MyEnv, self).__init__(**kwargs)
        self.para = para


tgt1 = '<geom axisangle="0 1 0 -3.8" name="bthigh" pos=".1 0 -.13" size="0.046 .145" type="capsule"/>'
tgt2 = '<geom axisangle="0 1 0 .52" name="fthigh" pos="-.07 0 -.12" size="0.046 .133" type="capsule"/>'
tgt_size = 'size'

tgt1_pos = 3707 + 7
tgt2_pos = 4594 + 7

# 修改为 Windows 风格路径
site_packages_dir = 'D:\\Anaconda\\envs\\py3.8\\Lib\\'

asset_path = site_packages_dir + 'site-packages\\gym\\envs\\mujoco\\assets\\'
ori_xml_path = asset_path + 'half_cheetah.xml'


def generate_xml(para):
    xml_file = 'half_cheetah' + ("%.2f" % para)[2:] + '.xml'
    xml_file_path = asset_path + xml_file
    if not os.path.exists(xml_file_path):
        with open(ori_xml_path, 'r') as f:
            # We modify the length of the thigh
            s = f.read()
        bthigh_length = round(143 + (para - 0.5) * 85)
        fthigh_length = round(133 + (para - 0.5) * 65)
        s_lst = list(s)
        s_lst[tgt1_pos: tgt1_pos + 3] = list(str(bthigh_length))
        s_lst[tgt2_pos: tgt2_pos + 3] = list(str(fthigh_length))
        s_new = ''.join(s_lst)
        with open(xml_file_path, 'w') as f_w:
            f_w.write(s_new)
    return xml_file


DEFAULT_CAMERA_CONFIG = {
    'distance': 4.0,
}


class MyHalfCheetahEnv(mujoco_env.MujocoEnv, utils.EzPickle, MyEnv):
    def __init__(self,
                 para=0.5,
                 forward_reward_weight=1.0,
                 ctrl_cost_weight=0.1,
                 reset_noise_scale=0.1,
                 exclude_current_positions_from_observation=True):
        MyEnv.__init__(self, para)
        para = round(para, 2)
        xml_file = generate_xml(para)
        utils.EzPickle.__init__(**locals())

        self.time = 0

        self._forward_reward_weight = forward_reward_weight

        self._ctrl_cost_weight = ctrl_cost_weight

        self._reset_noise_scale = reset_noise_scale

        self._exclude_current_positions_from_observation = (
            exclude_current_positions_from_observation)

        mujoco_env.MujocoEnv.__init__(self, xml_file, 5)

    def control_cost(self, action):
        control_cost = self._ctrl_cost_weight * np.sum(np.square(action))
        return control_cost

    def step(self, action):
        x_position_before = self.sim.data.qpos[0]
        self.do_simulation(action, self.frame_skip)
        x_position_after = self.sim.data.qpos[0]
        x_velocity = ((x_position_after - x_position_before)
                      / self.dt)

        ctrl_cost = self.control_cost(action)

        forward_reward = self._forward_reward_weight * x_velocity

        observation = self._get_obs()
        reward = forward_reward - ctrl_cost

        self.time += 1
        if self.time >= 1000:
            done = True
            self.time = 0
        else:
            done = False
        info = {
            'x_position': x_position_after,
            'x_velocity': x_velocity,

            'reward_run': forward_reward,
            'reward_ctrl': -ctrl_cost
        }

        return observation, reward, done, info

    def _get_obs(self):
        position = self.sim.data.qpos.flat.copy()
        velocity = self.sim.data.qvel.flat.copy()

        if self._exclude_current_positions_from_observation:
            position = position[1:]

        observation = np.concatenate((position, velocity)).ravel()
        return observation

    def reset_model(self):
        noise_low = -self._reset_noise_scale
        noise_high = self._reset_noise_scale

        self.time = 0

        qpos = self.init_qpos + self.np_random.uniform(
            low=noise_low, high=noise_high, size=self.model.nq)
        qvel = self.init_qvel + self._reset_noise_scale * self.np_random.randn(
            self.model.nv)

        self.set_state(qpos, qvel)

        observation = self._get_obs()
        return observation

    def viewer_setup(self):
        for key, value in DEFAULT_CAMERA_CONFIG.items():
            if isinstance(value, np.ndarray):
                getattr(self.viewer.cam, key)[:] = value
            else:
                setattr(self.viewer.cam, key, value)

In [12]:
# 创建两个不同的半猎豹环境
env1 = MyHalfCheetahEnv(para=0.1)  # 较短的thigh参数
env2 = MyHalfCheetahEnv(para=0.5)  
env3 = MyHalfCheetahEnv(para=0.9)  # 较长的thigh参数

In [13]:
from stable_baselines3 import PPO

In [14]:
model = PPO.load("D:/Study/RL_Project/rl-baselines3-zoo-1.7.0/logs/ppo/HalfCheetah-v3_1/HalfCheetah-v3.zip")

D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: an integer is required (got type bytes)
  warnings.warn(
D:\Anaconda\envs\py3.8\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:95: UserWarning: You loaded a model that was trained using OpenAI Gym. We strongly recommend tra

In [15]:
from stable_baselines3.common.evaluation import evaluate_policy

In [17]:
print(evaluate_policy(model, env1, n_eval_episodes= 10))
print(evaluate_policy(model, env1, n_eval_episodes= 10))
print(evaluate_policy(model, env1, n_eval_episodes= 10))
print()
print(evaluate_policy(model, env2, n_eval_episodes= 10))
print(evaluate_policy(model, env3, n_eval_episodes= 10))

(-1077.4177192560194, 48.68326364451065)
(-1067.0930099896389, 70.50206821549904)
(-1024.6321264254104, 64.0744645354474)

(-1027.258823139091, 59.33840806605491)
(-1104.8209268240694, 59.18149967074889)


In [18]:
asset_path

'D:\\Anaconda\\envs\\py3.8\\Lib\\site-packages\\gym\\envs\\mujoco\\assets\\'

In [19]:
# 定义文件名
file1_path = os.path.join(asset_path, 'half_cheetah10.xml')
file2_path = os.path.join(asset_path, 'half_cheetah90.xml')

def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def compare_files(file1_content, file2_content):
    differences = []
    file1_lines = file1_content.splitlines()
    file2_lines = file2_content.splitlines()
    
    for i, (line1, line2) in enumerate(zip(file1_lines, file2_lines), 1):
        if line1 != line2:
            differences.append((i, line1, line2))
    
    return differences

# 读取文件内容
file1_content = read_file(file1_path)
file2_content = read_file(file2_path)

# 比较文件
differences = compare_files(file1_content, file2_content)

# 输出不同之处
if differences:
    print("Files have differences at the following lines:")
    for line_num, line1, line2 in differences:
        print(f"Line {line_num}:")
        print(f"File 1: {line1}")
        print(f"File 2: {line2}")
        print("")
else:
    print("Files are identical.")

Files have differences at the following lines:
Line 64:
File 1:         <geom axisangle="0 1 0 -3.8" name="bthigh" pos=".1 0 -.13" size="0.046 .109" type="capsule"/>
File 2:         <geom axisangle="0 1 0 -3.8" name="bthigh" pos=".1 0 -.13" size="0.046 .177" type="capsule"/>

Line 76:
File 1:         <geom axisangle="0 1 0 .52" name="fthigh" pos="-.07 0 -.12" size="0.046 .107" type="capsule"/>
File 2:         <geom axisangle="0 1 0 .52" name="fthigh" pos="-.07 0 -.12" size="0.046 .159" type="capsule"/>

